<a href="https://colab.research.google.com/github/JacobTumak/cap-comp215/blob/main/Lab2_3_NEO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

COMP 215 - LAB 2 (NEO)
----------------
#### Name: Jacob Tumak
#### Date: 2023-04-08

This lab exercise is mostly a review of strings, tuples, lists, dictionaries, and functions.

**Building on new concepts from lab 1**:
  * `datetime.date` objects represent a calendar date
  * *list comprehension* provides a compact way to represent map and filter algorithms

**New Python Concepts**:
  * *f-string* simplifies string formatting operations

As usual, the first code cell simply imports all the modules we'll be using...

In [1]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/)
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

You should register for your own API key, (but may use the DEMO_KEY to get started).

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [2]:
today = datetime.date.today()   # get a date object representing today's date
print(today, type(today))
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the stringg
print(formatted_date)

2023-04-13 <class 'datetime.date'>
Today is: 2023-04-13


### Make a query

Let's get some data from the NEO database...
Here's a query that gets the observation "feed" for today.
(Note: I hard-coded the date below to lock down the data for the lab - ideally used `today()` so the notebook is always up-to-date.)

In [3]:
API_KEY = 'DEMO_KEY'  # substitute your API key here

today = '2023-01-09'  #  Future enhancement:  str(datetime.date.today())   # Today's date as a string!
# Use an f-string here to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

response = requests.request("GET", url, headers={}, data={})

data = json.loads(response.text)  # recall json.loads for lab 1

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = data['element_count']
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')
pprint(neos)

12 Near Earth Objects found for 2023-01-09
[{'absolute_magnitude_h': 19.64,
  'close_approach_data': [{'close_approach_date': '2023-01-09',
                           'close_approach_date_full': '2023-Jan-09 13:59',
                           'epoch_date_close_approach': 1673272740000,
                           'miss_distance': {'astronomical': '0.0725378004',
                                             'kilometers': '10851500.434325148',
                                             'lunar': '28.2172043556',
                                             'miles': '6742809.7016491224'},
                           'orbiting_body': 'Earth',
                           'relative_velocity': {'kilometers_per_hour': '32387.333793389',
                                                 'kilometers_per_second': '8.9964816093',
                                                 'miles_per_hour': '20124.2297117866'}}],
  'estimated_diameter': {'feet': {'estimated_diameter_max': 2301.5744618737,
      

Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

## Exercise 1

In the code cell below, **re-write the accumulator loop above** as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [5]:
hazards = [item for item in neos if item['is_potentially_hazardous_asteroid']]
print(f'{len(hazards)} potentially hazardous asteroids identified.')

3 potentially hazardous asteroids identified.


## Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.

We can use this `id` to fetch complete orbital and close approach data for the NEO.

For example, this query fetches the complete data set for the first hazardous asteroid...


In [ ]:
id = hazards[0]['id']
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}/?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

pprint(data)

Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".

## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"
* use a loop if that is easier to understand - we will look at more compact algorithms to solve this problem in class.

Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [17]:
def get_nearest_miss(data_set, today=today):
  dates_dists = [(obj['close_approach_date'], obj['miss_distance']['kilometers']) for obj in data_set['close_approach_data']]
  return sorted(dates_dists)[0]

#**Lab 3 Work Below**

In the instructions you said "*This method also takes an Asteroid object as
input – the Asteroid to which the close approach data belongs*" referring to the CloseApproach classmethod. I feel like you were looking for something different than what I did, but I wasn't sure how to go about including the asteroid object in the CloseApproach class since the Asteroid and the CloseApproach Classes reference eachother so in order to build one you have to have the other already built. I also think I may have misunderstood what you were looking for.

This is what I came up with as a solution. Accessing the asteroid object through the closeapproach data like this: `Asteroid.close_approaches[list_index].asteroid` will return a corresponding asteroid object but with an empty `close_approaches` list.

In [80]:
from dataclasses import dataclass
@dataclass
class Asteroid:
  id: int
  name: str
  estimated_diameter: float
  potential_hazard: bool
  close_approaches: list

  @classmethod
  def from_NEO(cls, neo_id):

    url = f'https://api.nasa.gov/neo/rest/v1/neo/{str(neo_id)}?api_key=DEMO_KEY'
    response = requests.request("GET", url, headers={}, data={})
    asteroid_data = json.loads(response.text)

    id = int(neo_id)
    name = asteroid_data['name']
    estimated_diameter = asteroid_data['estimated_diameter']['kilometers']
    potential_hazard = asteroid_data['is_potentially_hazardous_asteroid']
    asteroid_obj = Asteroid(id, name, estimated_diameter, potential_hazard, [])
    close_approaches = [CloseApproach.from_record(record, asteroid_obj) for record in asteroid_data['close_approach_data']]
    return Asteroid(id, name, estimated_diameter, potential_hazard, close_approaches)

In [74]:
@dataclass
class CloseApproach:
  asteroid: Asteroid
  orbiting_body: str
  approach_date: datetime.datetime
  miss_distance: float #miss_distance in km
  relative_velocity: float #relative velocity in km/h

  @classmethod
  def from_record(cls, record, asteroid_obj):
    asteroid = asteroid_obj
    orbiting_body = record['orbiting_body']
    approach_date = datetime.datetime.strptime(record['close_approach_date'], '%Y-%m-%d')
    miss_distance = record['miss_distance']['kilometers']
    relative_velocity = record['relative_velocity']['kilometers_per_hour']
    return CloseApproach(asteroid, orbiting_body, approach_date, miss_distance, relative_velocity)

In [86]:
trial = Asteroid.from_NEO(data['id'])
trial.potential_hazard

True

In [84]:
trial.close_approaches[0].asteroid.close_approaches

[]